Bayes Rule Book:

https://www.bayesrulesbook.com/chapter-2.html

Materials from the Bayes Rule github:

https://github.com/bayes-rules/bayesrules

# Imports

In [1]:
import pyreadr, math
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from os.path import exists

# Fun with Beta distribution

In [2]:
fig = make_subplots(rows=3, cols=3, subplot_titles=('1,1', '1, 5', '1,10', '5,1', '5,5', '5,10', '10,1', '10,5', '10,10'))
for i, r in enumerate([1,5,10]):
    for j, c in enumerate([1,5,10]):
        fig.add_trace(ff.create_distplot([np.random.beta(r,c,10000)], ['distplot'], show_hist=False, show_rug=False)['data'][0], row=i+1, col=j+1)

fig.update_layout(showlegend=False, height= 3*200, title="Beta Distributions with various alpha, beta values")

fig


# Beta Prior

## Distribution

Book doesn't provide underlying dataset, but assumes a prior with $π ∼Beta(45,55)$

In [3]:
alpha = 45
beta  = 55
prior = np.random.beta(alpha, beta,10000)
fig = ff.create_distplot([prior], ['prior'], show_hist=False, show_rug=False)
fig

## Mean, Mode, Variance

- $E(\pi) = \frac{\alpha}{\alpha+\beta}$
- $Mode(\pi) = \frac{\alpha-1}{\alpha+\beta-2}$
- $Var(\pi) = \frac{\alpha\beta}{(\alpha+\beta)^2(\alpha+\beta+1)}$

In [4]:
prior_mean = alpha / (alpha+beta)
prior_mode = (alpha-1) / (alpha+beta-2)
prior_std = np.sqrt((alpha*beta)/((alpha+beta)**2 * (alpha+beta+1)))
prior_mean

0.45

In [32]:
fig = go.Figure(go.Scatter(ff.create_distplot([prior], ['prior'], show_hist=False, show_rug=False)['data'][0]))

# Add Mean
fig.add_annotation(text="Mean", x=prior_mean, y=8, showarrow=True)
fig.add_shape(type="line", x0=prior_mean, y0=0, x1=prior_mean, y1=8, line={'color':"Red",'width':2})

# Add Median
fig.add_annotation(text="Mode", x=prior_mode, y=6, showarrow=True)
fig.add_shape(type="line", x0=prior_mode, y0=0, x1=prior_mode, y1=8, line={'color':"Green",'width':2})

# Add single std to both sides
fig.add_shape(type="line", x0=prior_mean - prior_std, y0=0, x1=prior_mean - prior_std, y1=5, line={'color':"Black",'width':2})
fig.add_shape(type="line", x0=prior_mean + prior_std, y0=0, x1=prior_mean + prior_std, y1=5, line={'color':"Black",'width':2})


fig

# Binomial Likelihood

Trying out different likelihoods against our observation(from book) of $Y=3$

In [86]:
n = 50
likelihood_means = []
fig = make_subplots(rows=3, cols=3, subplot_titles=[f'p={n/10}' for n in range(1,10)])

for i in range(9):
    dist_data = np.random.binomial(n, (i+1)/10, size=1000)
    likelihood_means.append(np.mean(dist_data))
    fig.add_trace(
        ff.create_distplot([dist_data], [f'likelihood_{(i+1)/10}'], show_rug=False, show_hist=False)['data'][0],
        row = (i//3)+1, col= (i%3)+1
    )

fig.update_layout(shapes=[dict(type="line", xref=f"x{n}", yref=f"y{n}",x0=30, y0=0, x1=30, y1=0.1, line=dict(color=f"{'Red' if (n<4 or n>7) else 'Green'}",width=3)) for n in range(1,10)])
fig.update_layout(xaxis_range=[0,50], showlegend=False, height=3*200)
fig.update_layout({f'xaxis{n}_range':[0,50] for n in range(2,10)})

fig.show()

In [111]:
def n_choose_k(n,k):
    return np.math.factorial(n)/(np.math.factorial(k)*np.math.factorial(n-k))

def binomial_p(n, k, p):
    return n_choose_k(n,k) * p**k * (1-p)**(n-k)

likelihood = [binomial_p(50,30,n/100) for n in range(1,100)]

px.line(x=list(range(1,100)), y=likelihood)

From the above, we see the likelihoods that p = [0.1...0.3, 0.8,0.9] is basically none. P is likely in the range of 0.4...0.7, with the **most likely being 0.6**.

# Beta Posterior

Book provides posterior $~Beta(75,75)$

As previously taught, the beta-binomial has the convenient property that the posterior beta is parameterized with $Beta(\alpha+k, \beta+(n-k))$

In [145]:
go.Figure([
    go.Scatter(x=[i/100 for i in range(1,100)], y=np.array(likelihood)*50, name='likelihood'),
    go.Scatter(ff.create_distplot([prior], ['prior'], show_hist=False, show_rug=False, colors=['Orange'])['data'][0]),
    go.Scatter(ff.create_distplot([np.random.beta(75, 75, 1000)], ['posterior'], show_hist=False, show_rug=False, colors=['Green'])['data'][0])
])

# End of Ch. Additional beta/gamma math functions

$\Gamma(n)=(n-1)!$

In [165]:
def gamma(n):
    return np.math.factorial(n-1)

*PDF of Beta*
$Beta(\alpha, \beta)=\frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1}(1-x)^{\beta-1}$

In [170]:
def beta_pdf(a, b, x):
    c = gamma(a+b) / (gamma(a)*gamma(b))
    return c * (x**(a-1)) * ((1-x)**(b-1))

In [209]:
a = 25
b = 35
px.line(x=[x/100 for x in range(1,100)], y=[beta_pdf(a,b,x/100) for x in range(1,100)])

$Bin(n, p)={n \choose k} p^k(1-p)^{n-k}$

In [213]:
def n_choose_k(n,k):
    return np.math.factorial(n)/(np.math.factorial(k)*np.math.factorial(n-k))

def binomial_p(n, k, p):
    return n_choose_k(n,k) * p**k * (1-p)**(n-k)

def binomial_likelihood(n,k):
    return [binomial_p(n, k, p/100) for p in range(1, 100)]

px.line(x=[x/100 for x in range(1,100)], y=binomial_likelihood(50,30))